# RFSoC Introduction

In this series of notebooks, we will introduce the RFSoC hardware and RFSoC relevant transmitter frequency bands.

This will be done in three sections:
- An overview of the RFSoC device hardware
- Frequency bands and Spectrum
- Transmitter Safety

## RFSoC Device Overview

This first notebook will be focused around the archetecture of the RFSoC hardware. The RFSoC is a Zynq System on Chip (SoC) targeted specifically at RF applications. It features a Processing System (PS), Programmable Logic (PL) and specialist RF resources. 

(Diagram from notes would be ideal here)
(Picture of our RFSoC?)

The specialist RF resources included in the RFSoC are:
- Transmit Blocks: 8 or 16 very high rate RF Digital to Analogue Converters (RF-DACs). These blocks also include Digital Up Converters (DUCs) for increasing the sampling rate and performing modulation. 
- Receive blocks: 8 or 16 very high rate RF Analogue-to-Digital (RF-ADCs). These blocks feature Digital Down Converters (DDCs) for demodulating the received signal and then reducing the sampling rate.
- Soft Decision Forward Error Correction (SD-FEC) blocks: 8 blocks for FEC encoding and/or decoding. These blocks support FEC coding schemes that are commonly used in 4G and 5G mobile standards.

There are also hardened resources for high speed wired communications, in the form of GTY Transceiver blocks. These blocks are commonly required for backhaul in wireless networks, namely CPRI (Common Public Radio Interface) and GigE (Gigabit Ethernet).

A general overview of the RFSoC device family can be seen from the Xilinx product page:

https://www.xilinx.com/products/silicon-devices/soc/rfsoc.html#ProductTable

### Notable Features

The RFSoC device has a number of notable features which seperate it from a traditional radio transceiver.

**Single Chip Integration**

The RFSoC integrates DACs and ADCs into the same chip as the processing resources needed to implement a radio transceiver.  

Other solutions need at least 2 chips e.g. a Zynq/Zynq MPSoC for signal processing paired with a front end radio transceiver for modulating and demodulating the signal to and from its target RF Frequency. 

(Image from notes)

**High Sample Rate**

The RF-ADCs and RF-DACs are both capable of sampling at multiple GHz, providing a very wide RF bandwidth. The RFSoC can directly digitise an RF band from 0 to 2 GHz. 

(Want a simple demo in here of spectrum analyser, want to show this wide bandwidth in action.)

(Is a comparison between RFSoC and much narrower SDR chip possible?)


**Multiple Channels**

Each RFSoC device has multiple channels (8 or 16) of transmit and receive functionality. These can be used for:

- Multiple Input Multiple Output (MIMO) systems
- Beamforming/Beamsteering

MIMO introduces spatial diversity to improve the throughput achievable over a link, because there are multiple transmitters and multiple receivers involved, each with a different position in space.

(What can we do in the way of a MIMO/beamforming demo? Is there a notebook that demonstrates this?)

Aside from these multi-antenna systems, the 8 or 16 channels of the RFSoC can also be used to support multiple different transmit and receive paths of the same or different radio standards on the same chip. This compares to a conventional solution where several chips would be required. 

### PS Architecture

The RFSoC's Processing System (PS) includes several different types of processing resources. These can be used to implement the software based aspects of an SDR. These resources include:

- External Interfaces: local IoT devices, SDR backhaul, networked system control.
- Platform Management and security functions: system booting, power management, physical security, SDR comms security.
- Dual-core Arm Cortex-R5 real-time processor: low level, deterministic SDR system control and orchestration.
- PS local memories, external memory control: Software programs, SDR instructions and data
- Quad-core Arm Cortex-A53 applications processor: running operating system, SDR software applications

(PS diagram from notes)

The four Arm Cortex-A53 processors are capable of running a fully featured Operating System and software applications. In the examples in this course the PYNQ software framework is deployed on the applications processor. More details on PYNQ can be found here: http://www.pynq.io/

### PL Architecture 

The Programmable Logic of the RFSoC is equivalent to that of an FPGA. Most devices in the RFSoC family provide a standard set of resources, including:

- High speed, wired interfaces (GTY transceivers, PCIe, 150G Interlaken, 100G Ethernet)
- Configurable Logic Blocks (930K logic cells)
- DSP Slices (4272)
- Block RAM (38 Mb)
- Ultra RAM (22.5 Mb)

AXI PS-to-PL interconnections enable control and data to pass quickly between the software and hardware systems.

(PL Diagram from notes)

The PL is where the SDR hardware design will be created. The parallel, programmable nature of the PL is ideal for implementing Digital Signal Processing (DSP) for the physical layer of SDR transmitter and receivers. Hardware subsystems can be desigend for flexible oepration, programmed from software.

### Data Converters

There are two types of RF Data Converters on the RFSoC:

Transmit side: An RF-DAC and Digital Up-Converter (DUC). The Interpolators, NCO and sampling rate are programmable. 

(DAC Diagram)

Receive side: An RF-ADC and Digital Down-Converter (DDC), likewise programmable. 

(ADC Diagram)

The sampling rates for the RF-DACs and RF-ADCs are programmable, with maximum and minimum values depending on the RFSoC Generation. For Gen 1, the RF-DACs can operate at sample rates of up to 6.554 Gsps, and the RF-ADCs can operate at up to 4.096 Gsps.

The NCO frequency is programmable. When the standard operating mode is used, it represents the RF frequency that the baseband signal is modulated to, and demodulated from.

The Interpolators and decimators can be programmed to undertake rate changes of 1 (no rate change), 2, 4 and 8, respectively. Further rate interpolation or decimation can be implemented in the PL.

### Data Converter Hierarchy

**RF-ADCs as tiles**

RFSoC devices have either 8 or 16 RF-ADCs, grouped into tiles alongside mixers and decimators. The spec

There are two different tile configurations:

- Dual RF-ADC tiles: 2x RF-ADCs, 2x mixers, 2x decimators
- Quad RF-ADC tiles: 4x RF-ADCs, 4x mixers, 4x decimators

(Could create diagram(s) showing these configs)

Each RF-ADC tile includes its own Phase Locked Loop (PLL) for managing the sampling clock applied to the RF-ADCs. 

**Dual RF-ADC Tile**

A dual RF-ADC tile supports two RF input channels:

(Dual tile diagram)



**RF-DACs**

At the transmit side, the RFSoC contains RF Data Converters in the form of an RF-DAC and Digital Up-Converter (DUC). The Interpolators, NCO and sampling rate involved with this are programmable.

These components are arranged in a Quadrature Amplitude Modulation (QAM) architecture. That is, the RF-DAC takes In-Phase (I) and Quadrature (Q) components as a Complex input signal. This architecture can be seen here:

(QAM RF-DAC)

### SD-FEC Blocks